# 1. Introduction
Reference <a href='https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/'>Data Flair team</a>

### 1.1 Mel-frequency cepstrum <a href='https://en.wikipedia.org/wiki/Mel-frequency_cepstrum'>Link</a>

In sound processing, the mel-frequency cepstrum (MFC) is a representation of the short-term power spectrum of a sound, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency.

Mel-frequency cepstral coefficients (MFCCs) are coefficients that collectively make up an MFC.[1] They are derived from a type of cepstral representation of the audio clip (a nonlinear "spectrum-of-a-spectrum"). The difference between the cepstrum and the mel-frequency cepstrum is that in the MFC, the frequency bands are equally spaced on the mel scale, which approximates the human auditory system's response more closely than the linearly-spaced frequency bands used in the normal cepstrum. This frequency warping can allow for better representation of sound, for example, in audio compression.

### 1.2 Chroma <a href='https://en.wikipedia.org/wiki/Chroma_feature'>Link</a>

In music, the term chroma feature or chromagram closely relates to the twelve different pitch classes. Chroma-based features, which are also referred to as "pitch class profiles", are a powerful tool for analyzing music whose pitches can be meaningfully categorized (often into twelve categories) and whose tuning approximates to the equal-tempered scale. One main property of chroma features is that they capture harmonic and melodic characteristics of music, while being robust to changes in timbre and instrumentation.

### 1.3 Mel Spectrogram Frequency  <a href='https://librosa.org/doc/0.7.2/generated/librosa.feature.melspectrogram.html'>Link</a>

Compute a mel-scaled spectrogram.

If a spectrogram input S is provided, then it is mapped directly onto the mel basis mel_f by mel_f.dot(S).

If a time-series input y, sr is provided, then its magnitude spectrogram S is first computed, and then mapped onto the mel scale by mel_f.dot(S**power). By default, power=2 operates on a power spectrum.

# 2. Data Cleaning

## 2.1 Importing Libraries

In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pandas as pd

## 2.2 Feature Extraction

### Steps
1. Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:

        i) mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
        ii) chroma: Pertains to the 12 different pitch classes
        iii) mel: Mel Spectrogram Frequency

2. Open the sound file with soundfile.SoundFile using with-as so it’s automatically closed once we’re done. 
        Read from it and call it X. Also, get the sample rate. If chroma is True, get the Short-Time Fourier Transform of X.

3. Let result be an empty numpy array. Now, for each feature of the three, if it exists, make a call to the corresponding function from librosa.feature (eg- librosa.feature.mfcc for mfcc), and get the mean value. Call the function hstack() from numpy with result and the feature value, and store this in result. hstack() stacks arrays in sequence horizontally (in a columnar fashion). Then, return the result.

- stft : Short-time Fourier transform (STFT)
- hstack : Stack arrays in sequence horizontally (column wise).

### 2.2.1 Extracting Feature

In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

### 2.2.2 Label Meaning

In [4]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

### 2.2.3 Loading Data and Generating Train Test splits

In [5]:
#DataFlair - Load the data and extract features for each sound file
# def load_data(test_size=0.2):
#     x,y=[],[]
#     for file in glob.glob("Data/Actor_*/*.wav"):
#         file_name=os.path.basename(file)
#         emotion=emotions[file_name.split("-")[2]]
#         if emotion not in observed_emotions:
#             continue
#         feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
#         x.append(feature)
#         y.append(emotion)
#     return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
def load_data():
    x,y=[],[]
    for file in glob.glob("Data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        #print(emotion)
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return np.array(x),np.array(y)

In [7]:
X,Y=load_data()

In [8]:
df=pd.DataFrame(X)
df['label']=Y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,label
0,-537.004395,31.666399,-6.542092,4.567606,-7.387805,-13.684703,-17.860582,-11.828496,-0.225548,-11.868469,...,0.001073,0.001119,0.001249,0.000946,0.000919,0.001475,0.001867,0.001133,0.000950,fearful
1,-383.007202,9.420696,-26.200880,2.628421,-17.898153,-16.619091,-16.220095,-11.781564,-1.197635,-13.881688,...,0.004942,0.004713,0.003613,0.003791,0.003923,0.003090,0.002704,0.001470,0.001169,fearful
2,-606.809570,42.631618,1.542243,5.561805,-6.484704,-4.223588,-14.775700,-7.947878,-1.427364,-4.774474,...,0.000669,0.000448,0.000357,0.000393,0.000801,0.000465,0.000162,0.000173,0.000086,disgust
3,-555.672119,43.681725,7.100577,7.280244,-7.252925,-12.699544,-14.237121,-13.417918,0.311144,-10.281244,...,0.006884,0.006208,0.006897,0.006973,0.008390,0.007365,0.003125,0.003766,0.003333,disgust
4,-526.230225,30.396181,-10.101453,5.733372,-10.386668,-13.651133,-12.541819,-9.009237,0.964835,-7.355454,...,0.002502,0.002561,0.005435,0.005965,0.004992,0.003769,0.004138,0.002868,0.001951,happy


In [12]:
df.label.value_counts()

disgust    192
calm       192
fearful    192
happy      192
Name: label, dtype: int64

In [15]:
def score_to_numeric(x):
    if x=='calm':
        return 0
    if x=='happy':
        return 1
    if x=='fearful':
        return 2
    if x=='disgust':
        return 3

In [16]:
df['num_label'] = df['label'].apply(score_to_numeric)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,label,num_label
0,-537.004395,31.666399,-6.542092,4.567606,-7.387805,-13.684703,-17.860582,-11.828496,-0.225548,-11.868469,...,0.001119,0.001249,0.000946,0.000919,0.001475,0.001867,0.001133,0.000950,fearful,2
1,-383.007202,9.420696,-26.200880,2.628421,-17.898153,-16.619091,-16.220095,-11.781564,-1.197635,-13.881688,...,0.004713,0.003613,0.003791,0.003923,0.003090,0.002704,0.001470,0.001169,fearful,2
2,-606.809570,42.631618,1.542243,5.561805,-6.484704,-4.223588,-14.775700,-7.947878,-1.427364,-4.774474,...,0.000448,0.000357,0.000393,0.000801,0.000465,0.000162,0.000173,0.000086,disgust,3
3,-555.672119,43.681725,7.100577,7.280244,-7.252925,-12.699544,-14.237121,-13.417918,0.311144,-10.281244,...,0.006208,0.006897,0.006973,0.008390,0.007365,0.003125,0.003766,0.003333,disgust,3
4,-526.230225,30.396181,-10.101453,5.733372,-10.386668,-13.651133,-12.541819,-9.009237,0.964835,-7.355454,...,0.002561,0.005435,0.005965,0.004992,0.003769,0.004138,0.002868,0.001951,happy,1


In [17]:
df.drop('label', axis=1, inplace=True)

In [19]:
#df.to_csv('data_reduced.csv')

# Load Data

In [45]:
new=pd.read_csv('data_reduced.csv')
new=new.iloc[:,1:]
#new['label']=new['label'].apply(lambda x:x-1)
print(new.shape)
new.head()

(768, 181)


,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,num_label
0,-537.004395,31.666399,-6.542092,4.567606,-7.387805,-13.684703,-17.860582,-11.828496,-0.225548,-11.868469,...,0.001073,0.001119,0.001249,0.000946,0.000919,0.001475,0.001867,0.001133,0.000950,2
1,-383.007202,9.420696,-26.200880,2.628421,-17.898153,-16.619091,-16.220095,-11.781564,-1.197635,-13.881688,...,0.004942,0.004713,0.003613,0.003791,0.003923,0.003090,0.002704,0.001470,0.001169,2
2,-606.809570,42.631618,1.542243,5.561805,-6.484704,-4.223588,-14.775700,-7.947878,-1.427364,-4.774474,...,0.000669,0.000448,0.000357,0.000393,0.000801,0.000465,0.000162,0.000173,0.000086,3
3,-555.672119,43.681725,7.100577,7.280244,-7.252925,-12.699544,-14.237121,-13.417918,0.311144,-10.281244,...,0.006884,0.006208,0.006897,0.006973,0.008390,0.007365,0.003125,0.003766,0.003333,3
4,-526.230225,30.396181,-10.101453,5.733372,-10.386668,-13.651133,-12.541819,-9.009237,0.964835,-7.355454,...,0.002502,0.002561,0.005435,0.005965,0.004992,0.003769,0.004138,0.002868,0.001951,1


In [24]:
new['num_label'].value_counts()

3    192
2    192
1    192
0    192
Name: num_label, dtype: int64

### Train Test Split

In [25]:
x_train,x_test,y_train,y_test=train_test_split(new.iloc[:,:-1],new['num_label'], test_size=0.25, random_state=9)

In [26]:
x_train.shape

(576, 180)

## 3. Machine Learning Model

### 3.1. MLPClassifier

In [27]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [28]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [29]:
y_pred=model.predict(x_test)

In [30]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 77.08%


### 3.2 XGBClassifier

In [31]:
import xgboost as xgb
from xgboost import XGBClassifier
model = XGBClassifier() 
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [32]:
y_pred = model.predict(x_test) 
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.79%


### 3.3 SVM RBF Kernel

In [33]:
from sklearn.svm import SVC

In [34]:
svm = SVC(kernel='rbf', random_state=0, gamma=.01, C=1)
# Train the classifier
svm.fit(x_train, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [35]:
y_pred=svm.predict(x_test)

In [36]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 43.23%


## Deep Learning

Before continuing, you must scale the training and test data, Xtr and Xts . Compute the mean and std deviation of each feature in Xtr and create a new training data set, Xtr_scale , by subtracting the mean and dividing by the std deviation. Also compute a scaled test data set, Xts_scale using the mean and std deviation learned from the training data set.

In [37]:
np.unique(y_train)

array([0, 1, 2, 3])

In [46]:
Xtr_scale = (x_train- (np.mean(x_train,axis=0)))/np.std(x_train,axis=0) 
Xts_scale = (x_test- (np.mean(x_train,axis=0)))/np.std(x_train,axis=0)

In [47]:
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.layers import Dense, Activation 
import tensorflow.keras.backend as K

In [48]:
K.clear_session()
nin = x_train.shape[1] # dimension of input data
nh = 256 # number of hidden units
nout = 9 # number of outputs = 10 since there are 10 classes 
model = Sequential()
model.add(Dense(units=nh, input_shape=(nin,), activation='sigmoid', name='hidden')) 
model.add(Dense(units=nout, activation='sigmoid', name='output'))

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 256)               46336     
_________________________________________________________________
output (Dense)               (None, 9)                 2313      
Total params: 48,649
Trainable params: 48,649
Non-trainable params: 0
_________________________________________________________________


In [50]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc')>0.99):
            print("\nReached 99% val accuracy so cancelling training!") 
            self.model.stop_training = True

In [51]:
from tensorflow.keras import optimizers 
callbacks = myCallback()
opt = optimizers.Adam(lr=0.001) 
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
hist = model.fit(Xtr_scale, y_train, epochs=2000, batch_size=100, validation_data=(Xts_scale,y_test))#,callbacks=callbacks)

Epoch 1/2000
6/6 [==============================] - 0s 44ms/step - loss: 2.2007 - accuracy: 0.2517 - val_loss: 1.9294 - val_accuracy: 0.3958
Epoch 2/2000
6/6 [==============================] - 0s 20ms/step - loss: 1.7604 - accuracy: 0.4201 - val_loss: 1.6155 - val_accuracy: 0.3906
Epoch 3/2000
6/6 [==============================] - 0s 12ms/step - loss: 1.5215 - accuracy: 0.4444 - val_loss: 1.4505 - val_accuracy: 0.3958
Epoch 4/2000
6/6 [==============================] - 0s 12ms/step - loss: 1.3939 - accuracy: 0.4705 - val_loss: 1.3625 - val_accuracy: 0.4375
Epoch 5/2000
6/6 [==============================] - 0s 11ms/step - loss: 1.3192 - accuracy: 0.5260 - val_loss: 1.3070 - val_accuracy: 0.4167
Epoch 6/2000
6/6 [==============================] - 0s 10ms/step - loss: 1.2677 - accuracy: 0.5660 - val_loss: 1.2647 - val_accuracy: 0.4427
Epoch 7/2000
6/6 [==============================] - 0s 11ms/step - loss: 1.2267 - accuracy: 0.5538 - val_loss: 1.2274 - val_accuracy: 0.4323
Epoch 8/2000


Epoch 59/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.4727 - accuracy: 0.8385 - val_loss: 0.8397 - val_accuracy: 0.6771
Epoch 60/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.4664 - accuracy: 0.8542 - val_loss: 0.8358 - val_accuracy: 0.6927
Epoch 61/2000
6/6 [==============================] - 0s 11ms/step - loss: 0.4659 - accuracy: 0.8507 - val_loss: 0.8326 - val_accuracy: 0.6771
Epoch 62/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.4576 - accuracy: 0.8438 - val_loss: 0.8321 - val_accuracy: 0.6875
Epoch 63/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.4552 - accuracy: 0.8490 - val_loss: 0.8366 - val_accuracy: 0.6875
Epoch 64/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.4501 - accuracy: 0.8542 - val_loss: 0.8339 - val_accuracy: 0.6979
Epoch 65/2000
6/6 [==============================] - 0s 8ms/step - loss: 0.4484 - accuracy: 0.8507 - val_loss: 0.8323 - val_accuracy: 0.6875
Epoch 66/20

6/6 [==============================] - 0s 8ms/step - loss: 0.2926 - accuracy: 0.9010 - val_loss: 0.8577 - val_accuracy: 0.7240
Epoch 117/2000
6/6 [==============================] - 0s 12ms/step - loss: 0.2913 - accuracy: 0.8924 - val_loss: 0.8530 - val_accuracy: 0.7292
Epoch 118/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.2875 - accuracy: 0.9010 - val_loss: 0.8567 - val_accuracy: 0.7240
Epoch 119/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.2863 - accuracy: 0.9097 - val_loss: 0.8522 - val_accuracy: 0.7188
Epoch 120/2000
6/6 [==============================] - 0s 8ms/step - loss: 0.2849 - accuracy: 0.9062 - val_loss: 0.8597 - val_accuracy: 0.7292
Epoch 121/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.2841 - accuracy: 0.9010 - val_loss: 0.8527 - val_accuracy: 0.7292
Epoch 122/2000
6/6 [==============================] - 0s 11ms/step - loss: 0.2793 - accuracy: 0.9062 - val_loss: 0.8580 - val_accuracy: 0.7188
Epoch 123/2000
6/

6/6 [==============================] - 0s 11ms/step - loss: 0.1805 - accuracy: 0.9566 - val_loss: 0.9022 - val_accuracy: 0.7292
Epoch 174/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.1789 - accuracy: 0.9635 - val_loss: 0.9085 - val_accuracy: 0.7188
Epoch 175/2000
6/6 [==============================] - 0s 8ms/step - loss: 0.1788 - accuracy: 0.9670 - val_loss: 0.9103 - val_accuracy: 0.7240
Epoch 176/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.1770 - accuracy: 0.9653 - val_loss: 0.9139 - val_accuracy: 0.7292
Epoch 177/2000
6/6 [==============================] - 0s 12ms/step - loss: 0.1754 - accuracy: 0.9635 - val_loss: 0.9084 - val_accuracy: 0.7292
Epoch 178/2000
6/6 [==============================] - 0s 11ms/step - loss: 0.1728 - accuracy: 0.9670 - val_loss: 0.9128 - val_accuracy: 0.7292
Epoch 179/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.1728 - accuracy: 0.9618 - val_loss: 0.9164 - val_accuracy: 0.7188
Epoch 180/2000
6

6/6 [==============================] - 0s 8ms/step - loss: 0.1079 - accuracy: 0.9896 - val_loss: 0.9555 - val_accuracy: 0.7292
Epoch 231/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.1079 - accuracy: 0.9878 - val_loss: 0.9590 - val_accuracy: 0.7292
Epoch 232/2000
6/6 [==============================] - 0s 10ms/step - loss: 0.1054 - accuracy: 0.9913 - val_loss: 0.9586 - val_accuracy: 0.7396
Epoch 233/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.1054 - accuracy: 0.9896 - val_loss: 0.9566 - val_accuracy: 0.7292
Epoch 234/2000
6/6 [==============================] - 0s 13ms/step - loss: 0.1040 - accuracy: 0.9913 - val_loss: 0.9630 - val_accuracy: 0.7292
Epoch 235/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.1029 - accuracy: 0.9913 - val_loss: 0.9634 - val_accuracy: 0.7396
Epoch 236/2000
6/6 [==============================] - 0s 9ms/step - loss: 0.1023 - accuracy: 0.9931 - val_loss: 0.9605 - val_accuracy: 0.7396
Epoch 237/2000
6/6

...

6/6 [==============================] - 0s 10ms/step - loss: 1.1132e-04 - accuracy: 1.0000 - val_loss: 1.4265 - val_accuracy: 0.7656
Epoch 1745/2000
6/6 [==============================] - 0s 10ms/step - loss: 1.1098e-04 - accuracy: 1.0000 - val_loss: 1.4262 - val_accuracy: 0.7656
Epoch 1746/2000
6/6 [==============================] - 0s 9ms/step - loss: 1.1088e-04 - accuracy: 1.0000 - val_loss: 1.4279 - val_accuracy: 0.7656
Epoch 1747/2000
6/6 [==============================] - 0s 11ms/step - loss: 1.1030e-04 - accuracy: 1.0000 - val_loss: 1.4276 - val_accuracy: 0.7656
Epoch 1748/2000
6/6 [==============================] - 0s 9ms/step - loss: 1.1003e-04 - accuracy: 1.0000 - val_loss: 1.4280 - val_accuracy: 0.7656
Epoch 1749/2000
6/6 [==============================] - 0s 9ms/step - loss: 1.0954e-04 - accuracy: 1.0000 - val_loss: 1.4271 - val_accuracy: 0.7656
Epoch 1750/2000
6/6 [==============================] - 0s 9ms/step - loss: 1.0923e-04 - accuracy: 1.0000 - val_loss: 1.4270 - val_a

6/6 [==============================] - 0s 10ms/step - loss: 9.4584e-05 - accuracy: 1.0000 - val_loss: 1.4353 - val_accuracy: 0.7656
Epoch 1800/2000
6/6 [==============================] - 0s 10ms/step - loss: 9.4492e-05 - accuracy: 1.0000 - val_loss: 1.4359 - val_accuracy: 0.7656
Epoch 1801/2000
6/6 [==============================] - 0s 10ms/step - loss: 9.4101e-05 - accuracy: 1.0000 - val_loss: 1.4338 - val_accuracy: 0.7656
Epoch 1802/2000
6/6 [==============================] - 0s 10ms/step - loss: 9.4023e-05 - accuracy: 1.0000 - val_loss: 1.4340 - val_accuracy: 0.7656
Epoch 1803/2000
6/6 [==============================] - 0s 7ms/step - loss: 9.3567e-05 - accuracy: 1.0000 - val_loss: 1.4346 - val_accuracy: 0.7656
Epoch 1804/2000
6/6 [==============================] - 0s 9ms/step - loss: 9.3571e-05 - accuracy: 1.0000 - val_loss: 1.4368 - val_accuracy: 0.7708
Epoch 1805/2000
6/6 [==============================] - 0s 12ms/step - loss: 9.2948e-05 - accuracy: 1.0000 - val_loss: 1.4365 - val

6/6 [==============================] - 0s 10ms/step - loss: 8.0464e-05 - accuracy: 1.0000 - val_loss: 1.4406 - val_accuracy: 0.7760
Epoch 1855/2000
6/6 [==============================] - 0s 9ms/step - loss: 8.0316e-05 - accuracy: 1.0000 - val_loss: 1.4405 - val_accuracy: 0.7708
Epoch 1856/2000
6/6 [==============================] - 0s 10ms/step - loss: 8.0170e-05 - accuracy: 1.0000 - val_loss: 1.4408 - val_accuracy: 0.7708
Epoch 1857/2000
6/6 [==============================] - ETA: 0s - loss: 7.6775e-05 - accuracy: 1.00 - 0s 9ms/step - loss: 7.9944e-05 - accuracy: 1.0000 - val_loss: 1.4419 - val_accuracy: 0.7708
Epoch 1858/2000
6/6 [==============================] - 0s 10ms/step - loss: 7.9625e-05 - accuracy: 1.0000 - val_loss: 1.4419 - val_accuracy: 0.7708
Epoch 1859/2000
6/6 [==============================] - 0s 8ms/step - loss: 7.9318e-05 - accuracy: 1.0000 - val_loss: 1.4417 - val_accuracy: 0.7708
Epoch 1860/2000
6/6 [==============================] - 0s 9ms/step - loss: 7.9254e-05

6/6 [==============================] - 0s 14ms/step - loss: 6.8725e-05 - accuracy: 1.0000 - val_loss: 1.4478 - val_accuracy: 0.7708
Epoch 1910/2000
6/6 [==============================] - 0s 8ms/step - loss: 6.8461e-05 - accuracy: 1.0000 - val_loss: 1.4477 - val_accuracy: 0.7708
Epoch 1911/2000
6/6 [==============================] - 0s 11ms/step - loss: 6.8191e-05 - accuracy: 1.0000 - val_loss: 1.4484 - val_accuracy: 0.7708
Epoch 1912/2000
6/6 [==============================] - 0s 10ms/step - loss: 6.8040e-05 - accuracy: 1.0000 - val_loss: 1.4491 - val_accuracy: 0.7708
Epoch 1913/2000
6/6 [==============================] - 0s 7ms/step - loss: 6.7883e-05 - accuracy: 1.0000 - val_loss: 1.4494 - val_accuracy: 0.7708
Epoch 1914/2000
6/6 [==============================] - 0s 10ms/step - loss: 6.7696e-05 - accuracy: 1.0000 - val_loss: 1.4508 - val_accuracy: 0.7708
Epoch 1915/2000
6/6 [==============================] - 0s 11ms/step - loss: 6.7455e-05 - accuracy: 1.0000 - val_loss: 1.4503 - val

6/6 [==============================] - 0s 10ms/step - loss: 5.8652e-05 - accuracy: 1.0000 - val_loss: 1.4542 - val_accuracy: 0.7708
Epoch 1965/2000
6/6 [==============================] - 0s 9ms/step - loss: 5.8496e-05 - accuracy: 1.0000 - val_loss: 1.4540 - val_accuracy: 0.7708
Epoch 1966/2000
6/6 [==============================] - 0s 8ms/step - loss: 5.8464e-05 - accuracy: 1.0000 - val_loss: 1.4561 - val_accuracy: 0.7708
Epoch 1967/2000
6/6 [==============================] - 0s 12ms/step - loss: 5.8277e-05 - accuracy: 1.0000 - val_loss: 1.4542 - val_accuracy: 0.7708
Epoch 1968/2000
6/6 [==============================] - 0s 9ms/step - loss: 5.8024e-05 - accuracy: 1.0000 - val_loss: 1.4547 - val_accuracy: 0.7708
Epoch 1969/2000
6/6 [==============================] - 0s 8ms/step - loss: 5.7924e-05 - accuracy: 1.0000 - val_loss: 1.4559 - val_accuracy: 0.7708
Epoch 1970/2000
6/6 [==============================] - 0s 8ms/step - loss: 5.7655e-05 - accuracy: 1.0000 - val_loss: 1.4570 - val_ac